In [2]:
import numpy as np
import cv2 
import matplotlib.pyplot as plt
import pydicom as dicom
import os
from scipy import ndimage as ndi 
from skimage import morphology
from skimage.morphology import skeletonize
from skimage.feature import canny
from skimage.future import graph
from skimage.segmentation import clear_border
from scipy import stats
from skimage import filters

c:\users\user\appdata\local\programs\python\python37\lib\site-packages\dicom\__init__.py:53: UserWarning: 
This code is using an older version of pydicom, which is no longer 
maintained as of Jan 2017.  You can access the new pydicom features and API 
by installing `pydicom` from PyPI.
See 'Transitioning to pydicom 1.x' section at pydicom.readthedocs.org 
for more information.

  warnings.warn(msg)


In [3]:
data_path = 'C:/Users/user/Desktop/Python/Python_processing/TCGA-SARC/2-AXIAL CAP-13097'
output_path = 'C:/Users/user/Desktop/Python/nii/pointcloud'

In [4]:
def load_scan(path):
    slices = [dicom.read_file(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key = lambda x: int(x.InstanceNumber))
    try:
        slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
    except:
        slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
        
    for s in slices:
        s.SliceThickness = slice_thickness
        
    return slices

c:\users\user\appdata\local\programs\python\python37\lib\site-packages\skimage\morphology\misc.py:127: UserWarning: Only one label was provided to `remove_small_objects`. Did you mean to use a boolean array?
  warn("Only one label was provided to `remove_small_objects`. "


In [5]:
def get_pixels_hu(scans):
    image = np.stack([s.pixel_array for s in scans])
    # Convert to int16 (from sometimes int16), 
    # should be possible as values should always be low enough (<32k)
    image = image.astype(np.int16)

    # Set outside-of-scan pixels to 0
    # The intercept is usually -1024, so air is approximately 0
    image[image == -2000] = 0
    
    # Convert to Hounsfield units (HU)
    intercept = scans[0].RescaleIntercept
    slope = scans[0].RescaleSlope
    
    if slope != 1:
        image = slope * image.astype(np.float64)
        image = image.astype(np.int16)
        
    image += np.int16(intercept)
    
    return np.array(image, dtype=np.int16)

In [16]:
#Save 3D CT image
patient = load_scan(data_path)
imgs = get_pixels_hu(patient)
np.save(output_path + "/3D_CT_image.npy", imgs)

In [17]:
#Load 3D CT image
CT_array = np.load(output_path+'/3D_CT_image.npy')

In [18]:
def sample_stack(stack, rows=10, cols=10, start_with=1, show_every=1):
    fig,ax = plt.subplots(rows,cols,figsize=[12,12])
    for i in range(rows*cols):
        ind = start_with + i*show_every
        ax[int(i/rows),int(i % rows)].set_title('slice %d' % ind)
        ax[int(i/rows),int(i % rows)].imshow(stack[ind],cmap='gray')
        ax[int(i/rows),int(i % rows)].axis('off')
    plt.show()

(512, 512, 794)

In [22]:
def threshold(image, threshmin=None, threshmax=None, newval=0):
    n_image = np.ma.array(image.copy())
    mask = np.zeros(n_image.shape, dtype=bool)
    if threshmin is not None:
        mask |= (n_image < threshmin).filled(False)

    if threshmax is not None:
        mask |= (n_image > threshmax).filled(False)

    n_image[mask] = newval
    return n_image

(1064039, 3)
(2140146, 3)


In [23]:
def Threshold(image, threshmin=None, threshmax=None, newval=0):
    n_image = image.copy()
    mask = np.zeros(n_image.shape, dtype=bool)
    if threshmin is not None:
        mask |= (n_image < threshmin).filled(False)

    if threshmax is not None:
        mask |= (n_image > threshmax).filled(False)

    n_image[mask] = newval
    return n_image

array([[-0.94543844, -0.2693067 , -0.18335776],
       [-0.4319342 , -0.25916055, -0.8638684 ],
       [-0.17195933, -0.93254864, -0.31746337],
       ...,
       [ 0.92002803, -0.323905  ,  0.22053108],
       [ 0.88382983,  0.41708818, -0.21185432],
       [ 0.91111964,  0.27139732,  0.31016842]], dtype=float32)

In [404]:
def generate_markers(image):
    
    #image Binary
    r_binary = image > -700
    binary = image < -700
    spine_b = image > 300
    
    ret,thresh1 = cv2.threshold(image,-700,1,cv2.THRESH_BINARY)
    kernel = np.ones((5,5),np.uint8)
    erosion = cv2.erode(thresh1, kernel,iterations = 1)
    opening = cv2.morphologyEx(erosion, cv2.MORPH_OPEN, kernel)
    closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel)
    
    #remove
    #remove = ndi.label(r_binary)
    #opening = morphology.binary_opening(r_binary)
    #cleared = morphology.binary_closing(opening)
    
    #Boundary Image
    #clear = clear_border(binary)
    #remove = morphology.remove_small_objects(binary, 100)
    #delet = filters.threshold_li(r_binary)
    
    
    #erosion = morphology.binary_erosion(delet)

    return r_binary, binary, spine_b, closing

In [406]:
r_binary, binary, spine_b, closing = generate_markers(CT_array)

In [408]:
sample_stack(r_binary)

In [410]:
sample_stack(binary)

In [412]:
sample_stack(closing)

In [414]:
from skimage import measure

In [416]:
def make_mesh(image, threshold=-300, step_size=1):

    print ("Transposing surface")
    p = image.transpose(2,1,0)
    
    print ("Calculating surface")
    verts, faces, norm, val = measure.marching_cubes_lewiner(p, threshold, step_size=step_size, allow_degenerate=True) 
    return verts, faces

In [418]:
def resample(image, scan, new_spacing=[1,1,1]):
    # Determine current pixel spacing
    spacing = np.array([scan[0].SliceThickness, scan[0].PixelSpacing[0], scan[0].PixelSpacing[1]])

    resize_factor = spacing / new_spacing
    new_real_shape = image.shape * resize_factor
    new_shape = np.round(new_real_shape)
    real_resize_factor = new_shape / image.shape
    new_spacing = spacing / real_resize_factor
    
    image = scipy.ndimage.interpolation.zoom(image, real_resize_factor)
    
    return image, new_spacing

In [420]:
#print ("Shape before resampling\t", r_binary.shape)
#CT_array, spacing = resample(r_binary, patient, [1,1,1])
#print ("Shape after resampling\t", CT_array.shape)

In [422]:
#v,f = make_mesh(CT_array,0,2)

In [ ]:
#x = v[:,2]
#x = 505 - x
#v[:,2] = x

In [ ]:
from open3d import *
import copy

In [423]:
#skeleton_data = PointCloud()
#skeleton_data.points = Vector3dVector(v)

In [424]:
#write_point_cloud(output_path + "/surface_thresh_from_CT.ply", skeleton_data)
#skeleton_data = read_point_cloud(output_path + "/surface_thresh_from_CT.ply")
#draw_geometries([skeleton_data])

In [425]:
#skeleton_data.paint_uniform_color([0.85, 0.65, 0.12])
#draw_geometries([skeleton_data])

In [426]:
print ("Shape before resampling\t", closing.shape)
CT_array, spacing = resample(closing, patient, [1,1,1])
print ("Shape after resampling\t", CT_array.shape)

In [ ]:
v,f = make_mesh(CT_array,0,2)

In [ ]:
x = v[:,2]
x = 505 - x
v[:,2] = x

In [ ]:
skeleton_data = PointCloud()
skeleton_data.points = Vector3dVector(v)

In [ ]:
write_point_cloud(output_path + "/surface_erosion_from_CT.ply", skeleton_data)
skeleton_data = read_point_cloud(output_path + "/surface_erosion_from_CT.ply")
draw_geometries([skeleton_data])

In [ ]:
skeleton_data.paint_uniform_color([0.85, 0.65, 0.12])
draw_geometries([skeleton_data])

In [ ]:
#현재 등 침상 부분까지제거하고 바운더리 얻는 부분에서 에러가 발생해서 수정중입니다..